# E6 멋진 작사가 만들기

일시: 2020.01.26

# Step 1. 데이터 다운로드

- 파일을 다운로드 받아 압축을 풀어줍니다.

```
$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics
```

# Step 2. 데이터 읽어오기

- glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장합니다.

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러 개의 txt 파일을 모두 읽어서 raw_corpus에 담기.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I. LIFE.', '', '']


# Step 3. 데이터 정제

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜀.

    if idx > 9: break   # 10개 문장 확인.
        
    print(sentence)

I. LIFE.
I.
SUCCESS.


In [3]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 변경, 양쪽 공백 삭제.
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백 추가.
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백 패턴을 만나면 스페이스 1개로 치환.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환.

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장으로 필터링 확인.

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
        
    corpus.append(preprocess_sentence(sentence))

corpus[:10]

['<start> i . life . <end>',
 '<start> i . <end>',
 '<start> success . <end>',
 '<start> published in a masque of poets <end>',
 '<start> at the request of h . h . , the author s <end>',
 '<start> fellow townswoman and friend . <end>',
 '<start> success is counted sweetest <end>',
 '<start> by those who ne er succeed . <end>',
 '<start> to comprehend a nectar <end>',
 '<start> requires sorest need . <end>']

- ```tokenize()```함수로 데이터를 Tensor로 변환합니다.

In [5]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지 생성.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, # 전체 단어의 개수. 
        filters = ' ', # 별도 전처리 로직 없음.
        oov_token = "<unk>"  # out-of-vocabulary.
    )
    tokenizer.fit_on_texts(corpus) # 구축한 corpus로부터 Tokenizer가 사전을 자동구축.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축.
    tensor = tokenizer.texts_to_sequences(corpus)   
    # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드를 제공.
    # maxlen는 start와 end가 빠질 것을 감안하여 16개로 지정.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post', maxlen = 16, truncating = 'post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print(tensor.shape)

[[   2    5   20 ...    0    0    0]
 [   2    5   20 ...    0    0    0]
 [   2 2762   20 ...    0    0    0]
 ...
 [   2  240    1 ...    0    0    0]
 [   2   10  502 ...    0    0    0]
 [   2  129   21 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f9d36b066d0>
(175986, 16)


In [6]:
print(tensor[:3, :10]) # 확인용.

[[   2    5   20  102   20    3    0    0    0    0]
 [   2    5   20    3    0    0    0    0    0    0]
 [   2 2762   20    3    0    0    0    0    0    0]]


In [7]:
for idx in tokenizer.index_word: # 인덱스 확인.
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break 

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내 소스 문장을 생성.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성.

print(src_input[0])
print(tgt_input[0])

[  2   5  20 102  20   3   0   0   0   0   0   0   0   0   0]
[  5  20 102  20   3   0   0   0   0   0   0   0   0   0   0]


# Step 4. 평가 데이터셋 분리

- ```sklearn```모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리합니다.

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140788, 15)
Target Train: (140788, 15)


- 텐서로 ```tf.data.Dataset```객체를 생성합니다. ```tf.data.Dataset.from_tensor_slices()```메소드를 사용합니다. 

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(train_dataset)
print(val_dataset)

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>


# Step 5. 인공지능 만들기 

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512 # 조절가능
hidden_size = 2048 # 조절가능
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

- 모델의 최종 출력은 shape = (256, 15, 12001)입니다.

In [12]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 15, 12001), dtype=float32, numpy=
array([[[ 1.45721351e-05, -9.88059837e-05, -1.07282598e-04, ...,
         -4.75916750e-04,  2.47447606e-04, -1.42766567e-05],
        [-4.47218808e-05, -2.42553451e-05,  8.75681653e-05, ...,
         -8.53255682e-04,  2.36173713e-04, -1.17513744e-04],
        [-3.46969551e-04,  1.34194503e-04, -9.31650502e-05, ...,
         -1.15912838e-03,  4.54906753e-04,  4.14822804e-04],
        ...,
        [ 1.63266412e-03, -8.79908097e-04,  1.64310652e-04, ...,
         -5.90977725e-05,  3.00545042e-04, -4.41536286e-05],
        [ 2.06640363e-03, -1.39321969e-03,  3.14098783e-04, ...,
          2.72786448e-04,  3.28425056e-04, -4.02136357e-05],
        [ 2.43392796e-03, -1.82875548e-03,  5.05866134e-04, ...,
          5.78969892e-04,  3.38637707e-04,  8.01623028e-05]],

       [[ 1.45721351e-05, -9.88059837e-05, -1.07282598e-04, ...,
         -4.75916750e-04,  2.47447606e-04, -1.42766567e-05],
        [-1.10995818e-04,  6.68799476e-05, -1

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  20979712  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none'), optimizer=optimizer)

history = model.fit(train_dataset, epochs = 10, validation_data = val_dataset, verbose = 1) # 10번 시도에 val_loss 2.2 이하 만들기

Epoch 1/10
549/549 [==============================] - 247s 451ms/step - loss: 3.3208 - val_loss: 2.9469
Epoch 2/10
549/549 [==============================] - 240s 436ms/step - loss: 2.7718 - val_loss: 2.6915
Epoch 3/10
549/549 [==============================] - 242s 441ms/step - loss: 2.4730 - val_loss: 2.5090
Epoch 4/10
549/549 [==============================] - 243s 443ms/step - loss: 2.1855 - val_loss: 2.3766
Epoch 5/10
549/549 [==============================] - 246s 449ms/step - loss: 1.9096 - val_loss: 2.2773
Epoch 6/10
549/549 [==============================] - 244s 444ms/step - loss: 1.6544 - val_loss: 2.2106
Epoch 7/10
549/549 [==============================] - 247s 450ms/step - loss: 1.4329 - val_loss: 2.1733
Epoch 8/10
549/549 [==============================] - 249s 454ms/step - loss: 1.2514 - val_loss: 2.1630
Epoch 9/10
549/549 [==============================] - 267s 486ms/step - loss: 1.1143 - val_loss: 2.1734
Epoch 10/10
549/549 [==============================] - 275s 501m

# 평가하기

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 함. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   
        # 우리 모델이 예측한 마지막 단어가 새롭게 생성한 단어가 됨.

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여줌.
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

- 하이퍼파라미터 embedding_size = 256, hidden_size = 1024일 때, val_loss는 2.54였습니다. 

- 하이퍼파라미터 embedding_size = 512, hidden_size = 2048일 때, val_loss는 2.19였습니다. 결과는 아래와 같습니다. 

In [16]:
generate_text(model, tokenizer, init_sentence = "<start> i love", max_len = 20)

'<start> i love it when they call me big poppa <end> '

In [17]:
generate_text(model, tokenizer, init_sentence = "<start> hey", max_len = 20)

'<start> hey , hey , hey , bobby mcghee , yeah son of a preacher man talkin bout a rock '

In [18]:
generate_text(model, tokenizer, init_sentence = "<start> you", max_len = 20)

'<start> you know i m bad , i m bad you know it <end> '

In [19]:
generate_text(model, tokenizer, init_sentence = "<start> keep", max_len = 20)

'<start> keep on with the force don t stop <end> '

# 루브릭 평가

**1. 가사 텍스트 생성 모델이 정상적으로 동작하는가? 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?**

도출된 문장은 다음과 같습니다. 

```'<start> i love it when they call me big poppa <end>'```

하이퍼파라미터 embedding_size = 256, hidden_size = 1024로 두고 epoch를 1번 돌렸을 때 문장은 ```'<start> i love you <end>'```였습니다. 짧게 학습해서 도출한 문장보다는 확실히 길고 어휘가 다양해짐을 확인할 수 있었습니다. 문장 길이를 더 길게 둔다면 노래 한 곡을 쓰는 건 일도 아닐 것 같습니다. 

**2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가? 특수문자 제거, 토크나이저 생성, 패딩 처리 등의 과정이 빠짐없이 진행되었는가?**

전처리와 데이터셋 구성 과정은 체계적으로 진행되었다고 생각합니다. 아직 코드를 짜라고 하라면 못짜지면, 코드를 보면 대략 작동 흐름은 알겠습니다. 저 위 3가지 고려 사항보다 제가 이해하는 데에 시간이 많이 걸렸던 것은 ```tf.data.Dataset``` 객체 생성 부분이었습니다.      

저는 계속 ```tf.data.Dataset``` 객체를 생성 과정이 ```train_test_split()```로 데이터셋을 분리하는 과정보다 앞이라고 생각했습니다. 그러니 dataset을 기껏 만들어놨것만, 훈련 데이터와 시험 데이터로 분리하는 데에 사용하지 않아서 dataset를 어디에 써야 하는 것인지 고민을 많이 했습니다. 다행히 조원의 도움으로 ```tf.data.Dataset``` 객체를 생성 과정 데이터셋 분류 뒤에 일어나는 일임을 알게 되어 분류한 데이터셋을 텐서로 잘 바꿀 수 있었습니다.

**3. 텍스트 생성모델이 안정적으로 학습되었는가? 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?**

처음 embedding_size = 256, hidden_size = 1024로 두고 모델을 학습시켰을 때 val_loss는 2.54였습니다. 제한된 epoch 10번 안에서 val_loss를 낮추기 위해 embedding_size와 hidden_size를 각각 2배씩 상향 조정하여 embedding_size = 512, hidden_size = 2048로 두고 모델을 재학습시켰습니다. 결과적으로 val_loss는 2.19까지 내려가게 되었습니다. 

# 회고

일을 미뤄서 하면 안된다는 것을 배운 프로젝트였습니다. 나름 저번주 할 때는 굉장히 재밌었고, 또 응용해볼만한 좋은 노드라고 생각했습니다. 그리고 다른 것들보다 쉽다고 생각했습니다. 그게 큰 실수였던 것 같습니다. 제출시간이 다됐는데, epoch가 도는 시간을 미처 예상하지 못해서 모델을 채 돌리기도 전에 제출합니다! 모델 훈련 시간을 줄이는 방법... 앞으로는 연구해서 꼭 학습이 빨리 끝나는 모델을 만들겁니다.    

제출하고 다음날 미처 돌리지 못한 모델 학습을 진행시켰습니다. 밤새 컴퓨터 작동이 없어서 그런지 1 epoch를 도는 시간이 굉장히 짧았습니다. 어제 밤에는 1 eopch에 17분이 걸렸는데 오늘은 5분 정도 걸렸던 것 같습니다. 이로 컴퓨터에 적당한 휴식을 주는 것이 학습 시간을 줄일 수 있는 방법 중에 하나라는 것을 깨달았습니다.    

모델 학습을 끝내고 문장 생성을 해보니 문법에도 거의 맞는 문장들이 나와서 놀랐습니다. 더 다양한 가사를 학습시키면 어휘가 풍부해질지 궁금합니다. 개인적으로는 인도 노래를 학습시키고 싶은데, 노랫말 모으는 것도 일이고 문법에 맞는지 검사하는 것도 일일 것 같네요.